In [ ]:
import pickle

load_path = "/content/drive/MyDrive/processed_faces_final/sampled_frames.pkl"

with open(load_path, "rb") as f:
    data = pickle.load(f)

real_sampled = data["real"]
fake_sampled = data["fake"]

print(f"Loaded Real Frames: {len(real_sampled)}")
print(f"Loaded Fake Frames: {len(fake_sampled)}")


Loaded Real Frames: 10000
Loaded Fake Frames: 9988


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import cv2

class DeepfakeDataset(Dataset):
    def __init__(self, frame_paths, label, transform=None):
        self.frame_paths = frame_paths
        self.labels = [label] * len(frame_paths)
        self.transform = transform

    def __len__(self):
        return len(self.frame_paths)

    def __getitem__(self, idx):
        img_path = self.frame_paths[idx]

        # Albumentations works with NumPy arrays (BGR format for OpenCV)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB

        if self.transform:
            image = self.transform(image=image)["image"]  # Albumentations returns a dict

        return image, self.labels[idx]


In [ ]:
#splitting

from sklearn.model_selection import train_test_split
import os

def split_video_wise(frames, test_size=0.15, val_size=0.15):
    # Extract video folder name from path
    video_folders = list(set([os.path.dirname(p) for p in frames]))

    train_videos, temp_videos = train_test_split(video_folders, test_size=(test_size + val_size), random_state=42)
    val_videos, test_videos = train_test_split(temp_videos, test_size=(test_size/(test_size + val_size)), random_state=42)

    def filter_by_video(video_list):
        return [p for p in frames if os.path.dirname(p) in video_list]

    return filter_by_video(train_videos), filter_by_video(val_videos), filter_by_video(test_videos)

real_train, real_val, real_test = split_video_wise(real_sampled)
fake_train, fake_val, fake_test = split_video_wise(fake_sampled)

print(len(real_train), len(real_val), len(real_test))
print(len(fake_train), len(fake_val), len(fake_test))


7000 1500 1500
6988 1500 1500


In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    return running_loss / total, correct / total


In [ ]:
def validate_one_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    return running_loss / total, correct / total


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

def get_transforms(flip_p, brightness_p, noise_p, compression_p, dropout_p):
    train_transforms = A.Compose([
        A.Resize(224, 224),
        A.HorizontalFlip(p=flip_p),
        A.RandomBrightnessContrast(p=brightness_p),
        A.GaussNoise(p=noise_p),
        A.ImageCompression(quality_lower=30, quality_upper=100, p=compression_p),
        A.CoarseDropout(
            max_holes=1, max_height=30, max_width=30,
            min_holes=1, min_height=10, min_width=10, p=dropout_p
        ),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

    val_transforms = A.Compose([
        A.Resize(224, 224),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

    return train_transforms, val_transforms


In [ ]:
from torch.utils.data import DataLoader, ConcatDataset

def get_dataloaders(batch_size, train_transforms, val_transforms, real_train, fake_train, real_val, fake_val):
    train_dataset = ConcatDataset([
        DeepfakeDataset(real_train, 0, transform=train_transforms),
        DeepfakeDataset(fake_train, 1, transform=train_transforms)
    ])
    val_dataset = ConcatDataset([
        DeepfakeDataset(real_val, 0, transform=val_transforms),
        DeepfakeDataset(fake_val, 1, transform=val_transforms)
    ])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, val_loader


In [ ]:
def get_model(dropout_rate, num_classes=2, freeze_ratio=0.8):
    # Load pretrained Xception model
    model = timm.create_model('xception', pretrained=True)

    # Freeze lower layers to reduce computation
    num_layers = len(list(model.parameters()))
    freeze_until = int(num_layers * freeze_ratio)

    for i, param in enumerate(model.parameters()):
        if i < freeze_until:
            param.requires_grad = False

    # Replace final classification head
    in_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(in_features, 512),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(512, num_classes)
    )

    # Move model to appropriate device (GPU/CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    return model, device

In [ ]:
def objective(trial):
    # --- Hyperparameter suggestions ---
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-3)
    dropout_rate = trial.suggest_float("dropout_rate", 0.3, 0.6)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    flip_p = trial.suggest_float("flip_p", 0.3, 0.8)
    brightness_p = trial.suggest_float("brightness_p", 0.2, 0.6)
    noise_p = trial.suggest_float("noise_p", 0.1, 0.4)
    compression_p = trial.suggest_float("compression_p", 0.2, 0.5)
    dropout_p = trial.suggest_float("dropout_p", 0.1, 0.4)

    # --- Build everything using helper functions ---
    train_transforms, val_transforms = get_transforms(flip_p, brightness_p, noise_p, compression_p, dropout_p)
    train_loader, val_loader = get_dataloaders(batch_size, train_transforms, val_transforms, real_train, fake_train, real_val, fake_val)
    model, device = get_model(dropout_rate)

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, factor=0.5)
    criterion = nn.CrossEntropyLoss()

    best_val_acc = 0.0
    num_epochs = 3
    for epoch in range(num_epochs):
        print(f"\n🚀 Epoch [{epoch+1}/{num_epochs}] (Trial {trial.number + 1})")
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = validate_one_epoch(model, val_loader, criterion, device)
        scheduler.step(val_loss)

        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Val Loss:   {val_loss:.4f}, Val Acc:   {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc

    return best_val_acc


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 10.5 MB/s eta 0:00:00


In [ ]:
import optuna
# ===========================
# RUN OPTUNA STUDY
# ===========================
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print("Best hyperparameters:")
print(study.best_params)
print("Best validation accuracy:")
print(study.best_value)


[I 2025-10-18 17:26:17,050] A new study created in memory with name: no-name-7418c4da-6d46-4e97-8c1a-f900f3ac8cc4
/tmp/ipython-input-1049881781.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
/tmp/ipython-input-1049881781.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-3)
/tmp/ipython-input-765546707.py:10: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=30, quality_upper=100, p=compression_p),
/tmp/ipython-input-765546707.py:11: UserWarning: Argument(s) 'max_holes

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-cadene/xception-43020ad28.pth" to /root/.cache/torch/hub/checkpoints/xception-43020ad28.pth

🚀 Epoch [1/3] (Trial 1)
Train Loss: 0.4720, Train Acc: 0.7446
Val Loss:   0.2387, Val Acc:   0.8943

🚀 Epoch [2/3] (Trial 1)
Train Loss: 0.2579, Train Acc: 0.8829
Val Loss:   0.2229, Val Acc:   0.9040

🚀 Epoch [3/3] (Trial 1)


[I 2025-10-18 18:26:08,089] Trial 0 finished with value: 0.9073333333333333 and parameters: {'lr': 1.0347111436502708e-05, 'weight_decay': 1.9053836607902433e-05, 'dropout_rate': 0.3215435255636158, 'batch_size': 16, 'flip_p': 0.7149653104858829, 'brightness_p': 0.36913895775030114, 'noise_p': 0.33107810776432345, 'compression_p': 0.29388511875788154, 'dropout_p': 0.2459611101547788}. Best is trial 0 with value: 0.9073333333333333.


Train Loss: 0.1882, Train Acc: 0.9159
Val Loss:   0.2185, Val Acc:   0.9073

🚀 Epoch [1/3] (Trial 2)
Train Loss: 0.2240, Train Acc: 0.8925
Val Loss:   0.2807, Val Acc:   0.9167

🚀 Epoch [2/3] (Trial 2)
Train Loss: 0.1071, Train Acc: 0.9540
Val Loss:   0.5169, Val Acc:   0.8840

🚀 Epoch [3/3] (Trial 2)


[I 2025-10-18 18:38:48,535] Trial 1 finished with value: 0.9166666666666666 and parameters: {'lr': 0.0003923245096601874, 'weight_decay': 1.7578179016206945e-05, 'dropout_rate': 0.49826107453660406, 'batch_size': 64, 'flip_p': 0.45804404511299446, 'brightness_p': 0.5798043459818183, 'noise_p': 0.35152770705903036, 'compression_p': 0.3109706119792101, 'dropout_p': 0.3275152809651438}. Best is trial 1 with value: 0.9166666666666666.


...

Train Loss: 0.0742, Train Acc: 0.9672
Val Loss:   0.2249, Val Acc:   0.9210

🚀 Epoch [1/3] (Trial 9)
Train Loss: 0.2527, Train Acc: 0.8794
Val Loss:   0.2792, Val Acc:   0.8923

🚀 Epoch [2/3] (Trial 9)
Train Loss: 0.1154, Train Acc: 0.9515
Val Loss:   0.4960, Val Acc:   0.8700

🚀 Epoch [3/3] (Trial 9)


[I 2025-10-18 20:04:03,583] Trial 8 finished with value: 0.9073333333333333 and parameters: {'lr': 5.7816923559252956e-05, 'weight_decay': 3.896572747075895e-06, 'dropout_rate': 0.3214158733807664, 'batch_size': 16, 'flip_p': 0.32592136411531586, 'brightness_p': 0.5770552770112205, 'noise_p': 0.2612249019073033, 'compression_p': 0.4263864644169515, 'dropout_p': 0.2833366920808057}. Best is trial 5 with value: 0.9226666666666666.


Train Loss: 0.0892, Train Acc: 0.9623
Val Loss:   0.2667, Val Acc:   0.9073

🚀 Epoch [1/3] (Trial 10)
Train Loss: 0.5212, Train Acc: 0.7420
Val Loss:   0.3262, Val Acc:   0.8687

🚀 Epoch [2/3] (Trial 10)
Train Loss: 0.2485, Train Acc: 0.8951
Val Loss:   0.2405, Val Acc:   0.8987

🚀 Epoch [3/3] (Trial 10)


[I 2025-10-18 20:16:43,599] Trial 9 finished with value: 0.9053333333333333 and parameters: {'lr': 1.0454531008720144e-05, 'weight_decay': 3.653306260836255e-05, 'dropout_rate': 0.5753045502840166, 'batch_size': 64, 'flip_p': 0.7833194321625924, 'brightness_p': 0.25907842444548945, 'noise_p': 0.17685650253488194, 'compression_p': 0.3546127115115837, 'dropout_p': 0.12892138920468682}. Best is trial 5 with value: 0.9226666666666666.


Train Loss: 0.1567, Train Acc: 0.9349
Val Loss:   0.2305, Val Acc:   0.9053
Best hyperparameters:
{'lr': 6.784292459046897e-05, 'weight_decay': 0.00013858849651110877, 'dropout_rate': 0.313977730486625, 'batch_size': 16, 'flip_p': 0.6464796243208715, 'brightness_p': 0.4652172584114009, 'noise_p': 0.2542681263548501, 'compression_p': 0.400378458078973, 'dropout_p': 0.1912157613506834}
Best validation accuracy:
0.9226666666666666


In [ ]:
import json

save_path = "/content/drive/MyDrive/processed_faces_final/best_params.json"  # you can change the folder name
with open(save_path, "w") as f:
    json.dump(study.best_params, f, indent=4)

print(f"✅ Best parameters saved to: {save_path}")


✅ Best parameters saved to: /content/drive/MyDrive/processed_faces_final/best_params.json


In [ ]:
import json

# Path where you saved the file
load_path = "/content/drive/MyDrive/processed_faces_final/best_params.json"

# Load parameters from JSON
with open(load_path, "r") as f:
    best_params = json.load(f)

print("Loaded best hyperparameters:", best_params)


Loaded best hyperparameters: {'lr': 6.784292459046897e-05, 'weight_decay': 0.00013858849651110877, 'dropout_rate': 0.313977730486625, 'batch_size': 16, 'flip_p': 0.6464796243208715, 'brightness_p': 0.4652172584114009, 'noise_p': 0.2542681263548501, 'compression_p': 0.400378458078973, 'dropout_p': 0.1912157613506834}


In [ ]:
def build_model(num_classes=2, freeze_ratio=0.8):
    # Load pretrained model
    model = timm.create_model('xception', pretrained=True)

    # Freeze lower layers
    num_layers = len(list(model.parameters()))
    freeze_until = int(num_layers * freeze_ratio)

    for i, param in enumerate(model.parameters()):
        if i < freeze_until:
            param.requires_grad = False

    # Replace final classification head
    in_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(in_features, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, num_classes)
    )

    return model

In [ ]:

# ===========================
# TRAIN FINAL MODEL WITH BEST PARAMS
# ===========================

# Reuse the same helper functions for final training
train_transforms, val_transforms = get_transforms(
    best_params["flip_p"],
    best_params["brightness_p"],
    best_params["noise_p"],
    best_params["compression_p"],
    best_params["dropout_p"]
)

train_loader, val_loader = get_dataloaders(
    best_params["batch_size"],
    train_transforms,
    val_transforms,
    real_train, fake_train, real_val, fake_val
)

model, device = get_model(best_params["dropout_rate"])

optimizer = optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, factor=0.5)

num_epochs = 10
for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate_one_epoch(model, val_loader, criterion, device)
    scheduler.step(val_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")



/tmp/ipython-input-765546707.py:10: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=30, quality_upper=100, p=compression_p),
/tmp/ipython-input-765546707.py:11: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width' are not valid for transform CoarseDropout
  A.CoarseDropout(


Epoch [1/10] - Train Acc: 0.8477, Val Acc: 0.8767
Epoch [2/10] - Train Acc: 0.9136, Val Acc: 0.8920
Epoch [3/10] - Train Acc: 0.9272, Val Acc: 0.8870
Epoch [4/10] - Train Acc: 0.9353, Val Acc: 0.8693
Epoch [5/10] - Train Acc: 0.9429, Val Acc: 0.8827
Epoch [6/10] - Train Acc: 0.9514, Val Acc: 0.8920
Epoch [7/10] - Train Acc: 0.9514, Val Acc: 0.8873
Epoch [8/10] - Train Acc: 0.9530, Val Acc: 0.8723
Epoch [9/10] - Train Acc: 0.9579, Val Acc: 0.8947
Epoch [10/10] - Train Acc: 0.9586, Val Acc: 0.8957


In [ ]:
import torch

save_path = "/content/drive/MyDrive/processed_faces_final/best_xception_optuna.pth"
torch.save(model.state_dict(), save_path)

print(f"✅ Model saved to: {save_path}")


✅ Model saved to: /content/drive/MyDrive/processed_faces_final/best_xception_optuna.pth
